In [55]:
import platform
platform.system()
import json
import pandas as pd 
import pydeck as pdk
import requests
import polyline
import googlemaps
import datetime
import time

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

#import pycurl

In [3]:
f = open('claves.json')
claves = json.load(f)

In [4]:
gmaps = googlemaps.Client(key=claves['GM'])


In [5]:
today = datetime.datetime.today()

In [6]:
def geocode_result(addr): 
    aa = gmaps.geocode(region='jm', address =  addr)
    return aa

In [7]:
def latlng(geocodeResult):
    GMlat= geocodeResult[0]['geometry']['location']['lat']
    GMlng= geocodeResult[0]['geometry']['location']['lng']
    
    return [GMlng, GMlat]

In [8]:
# reads in some local addresses
AddrDF=pd.read_csv("/Users/dominic.moss/Downloads/itel Dropoffs - Addr(1).csv")
AddrDF.head()

,WKT,UserID,firstname,lastname,StreetAddress,City,ScheduleDate,StartTime,EndTime,Lng,Lat
0,POINT (-77.8996807 18.4465605),421494,Michelle,Nelson,Granville,Falmouth P.O.,NaN,44282,0.4166666667,-77.899681,18.446561
1,POINT (-77.9225259 18.4757184),343073,Valrie,Walker,Paradise Norwood,Montego Bay,44282,0.2916666667,0.6666666667,-77.922526,18.475718
2,POINT (-78.0396129 18.2606101),523633,Chantel,Hemans,Whithorn Dist,Whithorn PA,NaN,44282,0.2916666667,-78.039613,18.260610
3,POINT (-78.2025913 18.4088304),647590,Phillicia,Gatchair,Cacoon District,Dias,44282,0.2916666667,0.6666666667,-78.202591,18.408830
4,POINT (-77.8989565 18.4792783),388912,Sasha-Lee,Dixon,Salt Spring,montego bay,44282,0.2916666667,0.6666666667,-77.898956,18.479278


In [9]:
Kgnpolygon = Polygon([ (-76.8994512,17.9246943),
 (-76.8341691,17.9749136),
 (-76.8156297,17.9605441),
 (-76.7177795,17.9562141),
 (-76.7334436,18.0202162),
 (-76.7331864,18.0281927),
 (-76.7512108,18.0361909),
 (-76.7738701,18.0596939),
 (-76.786498,18.0742406),
 (-76.7816915,18.0886008),
 (-76.7985143,18.1101389),
 (-76.8222036,18.108181),
 (-76.8462361,18.0732614),
 (-76.8719854,18.060532),
 (-76.8664922,18.0285412),
 (-76.9015111,18.0118906),
 (-76.9276036,18.0229912),
 (-76.9523229,18.0321323),
 (-76.981162,18.0494338),
 (-76.9997014,18.0239706),
 (-77.0072545,17.9968711),
 (-76.9986715,17.963889),
 (-76.9499196,17.9811972),
 (-76.8994512,17.9246943)])

In [10]:
##sample  polygon code
#point = Point(0.5, 0.5)
#polygon = Polygon([(0, 0), (0, 1), (1, 1), (1, 0)])
#print(polygon.contains(point))
#

In [11]:
AddrDF['KgnCheck'] = None 
for index, row in AddrDF.iterrows():
    point = Point(row['Lng'],row['Lat'])
    check = Kgnpolygon.contains(point)
    AddrDF.loc[index, 'KgnCheck'] = check


In [12]:
#AddrDF['KgnCheck'].describe()

In [13]:
#AddrDF[AddrDF['KgnCheck']]= True
AddrDF
AddrDFKgn = AddrDF[AddrDF['KgnCheck'] == True] 
AddrDFKgnSample = AddrDFKgn.head(70)

In [14]:
AddrDF

,WKT,UserID,firstname,lastname,StreetAddress,City,ScheduleDate,StartTime,EndTime,Lng,Lat,KgnCheck
0,POINT (-77.8996807 18.4465605),421494,Michelle,Nelson,Granville,Falmouth P.O.,NaN,44282,0.4166666667,-77.899681,18.446561,False
1,POINT (-77.9225259 18.4757184),343073,Valrie,Walker,Paradise Norwood,Montego Bay,44282,0.2916666667,0.6666666667,-77.922526,18.475718,False
2,POINT (-78.0396129 18.2606101),523633,Chantel,Hemans,Whithorn Dist,Whithorn PA,NaN,44282,0.2916666667,-78.039613,18.260610,False
3,POINT (-78.2025913 18.4088304),647590,Phillicia,Gatchair,Cacoon District,Dias,44282,0.2916666667,0.6666666667,-78.202591,18.408830,False
4,POINT (-77.8989565 18.4792783),388912,Sasha-Lee,Dixon,Salt Spring,montego bay,44282,0.2916666667,0.6666666667,-77.898956,18.479278,False
...,...,...,...,...,...,...,...,...,...,...,...,...
964,POINT (-76.7571202 17.9737203),594116,Shannoya,Nelson,Pleasant Heights,Rockfort,Kingston 2,44283,0.4583333333,-76.757120,17.973720,True
965,POINT (-76.7981869 18.0423156),641365,Okeeno,Gayle,99 Clifton Way Kingston 8,NaN,44283,0.3333333333,0.6875,-76.798187,18.042316,True
966,POINT (-76.7820968 18.0000916),631473,Racquel,Roach,28 Swallowfield Road,Kingston 5,NaN,44282,0.4583333333,-76.782097,18.000092,True
967,POINT (-76.7820968 18.0000916),631473,Racquel,Roach,28 Swallowfield Road,Kingston 5,NaN,44283,0.4583333333,-76.782097,18.000092,True


In [15]:
bounds2 = {
    'northeast' : '18.608803,-75.976979',
'southwest':'17.583947,-78.459537'
    
    
}

In [16]:
# geocode test
ab = gmaps.geocode(region='jm', address =  '1a Braemar ave', bounds = bounds2)

In [17]:
Kgn_Samp_List = ['Papine',
'Sovereign centre',
'Half way tree bus terminal',
'Knutsford Boulevard',
'Sabina park',
'Megamart Kingston',
'Pricesmart',
'Manor park plaza',
'Wolmers Prep',
'Vaz Prep',
'St andrew prep school',
'Peter and paul prep',
'Campion College ',
'Hopefield prep',
'Liguanea prep',
'Dunrobin primary ',
'Rockfort police station',
'Harbour view ',
'Queen''s school',
'St richard''s prep',
'Mona Prep',
'UWI ',
'AC hotel ',
'Devon house',
'Jamaica Pegasus hotel ',
'Azan''s supercentre',
'Fontana Waterloo ',
'Macau gaming ',
'Chilitos',
'Oaklands',
'Lindsey crescent apartment',
'Liguanea ave ',
'Paddington terrace',
'Windsor ave ',
'Shortwood Ave ',
'Burbank ave ',
'K&k mini mart ']

In [18]:
#AddrDFKgn.to_csv("/Users/dominic.moss/Downloads/kgnAddr.csv")

In [19]:
dire = gmaps.directions(origin = 'Shortwood Ave, kingston ', destination = 'Paddington terrace, kingston', region = 'JM' )

In [20]:
#pl = dire[0]['overview_polyline']['points']

In [23]:
#plD = polyline.decode(pl)
#list_of_lists = [list(tup) for tup in plD]
#LoL =  [list(reversed(tup)) for tup in list_of_lists ]
#LoL
#    locals()["LoL"+str(x)] =  [list(reversed(tup)) for tup in list_of_lists ]
#    locals()['dfRoute'+str(x)] = pd.DataFrame(response2['routes'][x]['steps'])   

In [24]:
dfHeartbeats = pd.DataFrame(columns = [
'user_id'
,'user_role' 
,'timestamp'
,'latitude'
,'longitude'
,'speed'
]
                           )

In [25]:
for x in range(len(Kgn_Samp_List)):
    #print (Kgn_Samp_List[x])
    #print (AddrDFKgn.iloc[x]['Lat'])
    #print (AddrDFKgn.iloc[x]['Lng'])
    locals()["dire"+str(x)] = gmaps.directions(origin = [AddrDFKgn.iloc[x]['Lat'],AddrDFKgn.iloc[x]['Lng']]  , destination = Kgn_Samp_List[x] +', kingston', region = 'JM', mode = 'driving'  )
    
    
    

In [26]:
for x in range(len(Kgn_Samp_List)):    
    locals()["dire"+str(x)]
    plD = polyline.decode(locals()["dire"+str(x)][0]['overview_polyline']['points'])
    list_of_lists = [list(tup) for tup in plD]
    locals()["LoL"+str(x)] =  [list(reversed(tup)) for tup in list_of_lists ]


In [27]:
#dfHeartbeats

In [28]:
tsStart=  1669980219
tsinit=  1669980219
for x in range(len(Kgn_Samp_List)):  
    index = x+1
    user_id = x+1
    tsStart = tsinit
    for y in range(len(locals()["LoL"+str(x)])):
        dfHeartbeats = pd.concat(objs = [dfHeartbeats, pd.DataFrame({'user_id' :user_id
                                   ,'user_role'  : 'driver'
                                ,'timestamp' : tsStart
                                ,'latitude' : locals()["LoL"+str(x)][y][1]
                                ,'longitude' : locals()["LoL"+str(x)][y][0]
                                ,'speed' : 25
                                
                                   }
        , index=[y+x])]
                            )
        tsStart = tsStart + 30 
    tsinit = tsinit + 60 


In [29]:
dfHeartbeats = dfHeartbeats.reset_index(drop=True)

In [30]:
#l
Colours = [
 '#faa61a'
,'#800080' 
,'#008080' 
,'#C0C0C0'
,'#FA8072'
,'#B22222'
,'#EEE8AA'
,'#F0E68C'
,'#00FA9A'
,'#000080' 


]

In [31]:
dfTrip = pd.DataFrame(columns = [
'user_id'
,'coordinates' 
,'timestamps'
,'color'
]
                           )


In [32]:
for x in range (37):
    cList = []
    tList= []
    for index,row in dfHeartbeats[dfHeartbeats['user_id'] == x+1].iterrows():
        tList.append(row['timestamp'])
        ctemp = [row['longitude'],row['latitude'] ]
        cList.append(ctemp)
    
    
    dfTrip = pd.concat(objs = [dfTrip, pd.DataFrame({'user_id' :x
    ,'coordinates'  : [cList]
    ,'timestamps' : [tList]
    ,'color':  '#faa61a'
                                
                                   }
        , index=[x])])

In [33]:
# Getting the unified route
for x in range(len(Kgn_Samp_List)):

    locals()["Tripdist"+str(x)] =     locals()["dire"+str(x)][0]['legs'][0]['distance']['value']
    locals()["pLat"+str(x)] =     locals()["dire"+str(x)][0]['legs'][0]['start_location']['lat']
    locals()["pLng"+str(x)] =     locals()["dire"+str(x)][0]['legs'][0]['start_location']['lng'] 
    locals()["dLat"+str(x)] =     locals()["dire"+str(x)][0]['legs'][0]['end_location']['lat']
    locals()["dLng"+str(x)] =     locals()["dire"+str(x)][0]['legs'][0]['end_location']['lng']


    

In [37]:
for x in range(len(Kgn_Samp_List)):
    locals()["ssP"+str(x)]= {'id': x*2   , 'description': 'Pickup - Passenger '+str(x), 'location': [locals()["pLng"+str(x)], locals()["pLat"+str(x)]], 'service':60*3 } # , 'time_windows': [[0,pickupWindow]]
    locals()["ssD"+str(x)]= {'id': (x*2)+1, 'description': 'Dropoff - Passenger '+str(x), 'location': [locals()["dLng"+str(x)], locals()["dLat"+str(x)]], 'service':60 } # , 'time_windows': [[0,dropoffWindow]]


In [38]:
for x in range(len(Kgn_Samp_List)):
    locals()["shipment"+str(x)]= {'pickup' : locals()["ssP"+str(x)], 'delivery': locals()["ssD"+str(x)] }

In [39]:
shipmentd = []
for x in range(len(Kgn_Samp_List)):
    shipmentd.append(locals()["shipment"+str(x)])

In [40]:
shipmentd

[{'pickup': {'id': 0,
   'description': 'Pickup - Passenger 0',
   'location': [-76.8868706, 17.9995857],
   'service': 180},
  'delivery': {'id': 1,
   'description': 'Dropoff - Passenger 0',
   'location': [-76.73577689999999, 18.0115061],
   'service': 60}},
 {'pickup': {'id': 2,
   'description': 'Pickup - Passenger 1',
   'location': [-76.7964533, 17.9664035],
   'service': 180},
  'delivery': {'id': 3,
   'description': 'Dropoff - Passenger 1',
   'location': [-76.76946319999999, 18.0206159],
   'service': 60}},
 {'pickup': {'id': 4,
   'description': 'Pickup - Passenger 2',
   'location': [-76.829804, 18.0103573],
   'service': 180},
  'delivery': {'id': 5,
   'description': 'Dropoff - Passenger 2',
   'location': [-76.7789731, 18.0063345],
   'service': 60}},
 {'pickup': {'id': 6,
   'description': 'Pickup - Passenger 3',
   'location': [-76.829804, 18.0103573],
   'service': 180},
  'delivery': {'id': 7,
   'description': 'Dropoff - Passenger 3',
   'location': [-76.7876304, 1

In [41]:
vehicle ={'id': 0, 'end': [dLng0, dLat0] }

In [42]:
data2 = {'vehicles': [vehicle], 'shipments' : shipmentd }

In [43]:
response = requests.post('http://solver.vroom-project.org'
                       ,headers={'Content-Type': 'application/json'}
                        ,data= json.dumps(data2),
                       )

In [44]:
response2  = response.json()


In [45]:
response2

{'code': 0,
 'summary': {'cost': 16446,
  'routes': 1,
  'unassigned': 0,
  'setup': 0,
  'service': 8880,
  'duration': 16446,
  'waiting_time': 0,
  'priority': 0,
  'distance': 163304,
  'violations': [],
  'computing_times': {'loading': 28, 'solving': 98, 'routing': 19}},
 'unassigned': [],
 'routes': [{'vehicle': 0,
   'cost': 16446,
   'setup': 0,
   'service': 8880,
   'duration': 16446,
   'waiting_time': 0,
   'priority': 0,
   'distance': 163304,
   'steps': [{'type': 'start',
     'location': [-76.9375615, 18.0119177],
     'setup': 0,
     'service': 0,
     'waiting_time': 0,
     'arrival': 0,
     'duration': 0,
     'violations': [],
     'distance': 0},
    {'type': 'pickup',
     'description': 'Pickup - Passenger 25',
     'location': [-76.9375615, 18.0119177],
     'id': 50,
     'setup': 0,
     'service': 180,
     'waiting_time': 0,
     'job': 50,
     'arrival': 0,
     'duration': 0,
     'violations': [],
     'distance': 0},
    {'type': 'pickup',
     'desc

In [47]:
l = polyline.decode(response2['routes'][0]['geometry'])
#dist = 163031

In [48]:
l

list_of_lists = [list(tup) for tup in l]

list_of_lists =  [list(reversed(tup)) for tup in list_of_lists ]

In [49]:
#cList = []
tListMatched= []
start2 = max(dfHeartbeats['timestamp']) + 1
for x in range(len(list_of_lists)):
        tListMatched.append(start2)
        start2 = start2 + 6 


In [50]:
dfTrip

,user_id,coordinates,timestamps,color
0,0,"[[-76.88687, 17.99959], [-76.88748, 17.99952],...","[1669980219, 1669980249, 1669980279, 166998030...",#faa61a
1,1,"[[-76.79645, 17.9664], [-76.79586, 17.96634], ...","[1669980279, 1669980309, 1669980339, 166998036...",#faa61a
2,2,"[[-76.8298, 18.01036], [-76.82957, 18.01021], ...","[1669980339, 1669980369, 1669980399, 166998042...",#faa61a
3,3,"[[-76.8298, 18.01036], [-76.82957, 18.01021], ...","[1669980399, 1669980429, 1669980459, 166998048...",#faa61a
4,4,"[[-76.78439, 17.98558], [-76.78331, 17.98535],...","[1669980459, 1669980489, 1669980519, 166998054...",#faa61a
5,5,"[[-76.78439, 17.98558], [-76.78567, 17.98586],...","[1669980519, 1669980549, 1669980579, 166998060...",#faa61a
6,6,"[[-76.74542, 17.99613], [-76.74491, 17.99666],...","[1669980579, 1669980609, 1669980639, 166998066...",#faa61a
7,7,"[[-76.74542, 17.99613], [-76.74491, 17.99666],...","[1669980639, 1669980669, 1669980699, 166998072...",#faa61a
8,8,"[[-76.81808, 18.05513], [-76.81818, 18.05506],...","[1669980699, 1669980729, 1669980759, 166998078...",#faa61a
9,9,"[[-76.82853, 18.03843], [-76.82843, 18.03829],...","[1669980759, 1669980789, 1669980819, 166998084...",#faa61a


In [51]:

    
    
dfTrip = pd.concat(objs = [dfTrip, pd.DataFrame({'user_id' :len(dfTrip)
    ,'coordinates'  : [list_of_lists]
    ,'timestamps' : [tListMatched]
    ,'color':  '#faa61a'
                                
                                   }
        , index=[len(dfTrip)])])

In [52]:
dfTrip

,user_id,coordinates,timestamps,color
0,0,"[[-76.88687, 17.99959], [-76.88748, 17.99952],...","[1669980219, 1669980249, 1669980279, 166998030...",#faa61a
1,1,"[[-76.79645, 17.9664], [-76.79586, 17.96634], ...","[1669980279, 1669980309, 1669980339, 166998036...",#faa61a
2,2,"[[-76.8298, 18.01036], [-76.82957, 18.01021], ...","[1669980339, 1669980369, 1669980399, 166998042...",#faa61a
3,3,"[[-76.8298, 18.01036], [-76.82957, 18.01021], ...","[1669980399, 1669980429, 1669980459, 166998048...",#faa61a
4,4,"[[-76.78439, 17.98558], [-76.78331, 17.98535],...","[1669980459, 1669980489, 1669980519, 166998054...",#faa61a
5,5,"[[-76.78439, 17.98558], [-76.78567, 17.98586],...","[1669980519, 1669980549, 1669980579, 166998060...",#faa61a
6,6,"[[-76.74542, 17.99613], [-76.74491, 17.99666],...","[1669980579, 1669980609, 1669980639, 166998066...",#faa61a
7,7,"[[-76.74542, 17.99613], [-76.74491, 17.99666],...","[1669980639, 1669980669, 1669980699, 166998072...",#faa61a
8,8,"[[-76.81808, 18.05513], [-76.81818, 18.05506],...","[1669980699, 1669980729, 1669980759, 166998078...",#faa61a
9,9,"[[-76.82853, 18.03843], [-76.82843, 18.03829],...","[1669980759, 1669980789, 1669980819, 166998084...",#faa61a


In [53]:


layer = pdk.Layer(
    "TripsLayer",
    dfTrip,
    get_path="coordinates",
    get_timestamps="timestamps",
    get_color=[228,87,86],
    opacity=0.2,
    width_min_pixels=2,
    rounded=True,
    trail_length=3000,
    current_time=tsStart,
)

#view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=11, bearing=0, pitch=45)
view_state = pdk.ViewState(latitude=18.014960873538982, longitude=-76.79001603118273, zoom=10)
# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.show()


DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"lat…

In [56]:
rp = min(dfHeartbeats['timestamp']) 
while rp <= max(tListMatched):
    layer.current_time = rp
    rp=rp+70
    r.update()
    time.sleep(0.02)